In [41]:
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions.regexp_replace
import org.apache.spark.ml.feature.CountVectorizer
import org.apache.spark.ml.feature.CountVectorizerModel
import org.apache.spark.ml.feature.IDF
import org.apache.spark.ml.feature.OneHotEncoderEstimator
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.feature.RegexTokenizer
import org.apache.spark.ml.feature.StopWordsRemover
import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.regression.LinearRegression

import org.apache.spark.ml.regression.{RandomForestRegressionModel, RandomForestRegressor}

import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions.regexp_replace
import org.apache.spark.ml.feature.CountVectorizer
import org.apache.spark.ml.feature.CountVectorizerModel
import org.apache.spark.ml.feature.IDF
import org.apache.spark.ml.feature.OneHotEncoderEstimator
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.ml.feature.RegexTokenizer
import org.apache.spark.ml.feature.StopWordsRemover
import org.apache.spark.ml.feature.StringIndexer
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.Pipeline
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.regression.LinearRegression
import org.apache.spark.ml.regression.{RandomForestRegressionModel, RandomForestRegressor}


In [2]:
// Import the dataFrame
import spark.implicits._
val dataset = spark.read.option("header", true)
.option("multiline",true).text("/home/martinez/git/ms2020bgd/SD701/airbnb/airbnb_paris.csv")
dataset.printSchema()




val toSave = dataset.withColumn("value", regexp_replace($"value", "\"\"",""))

val datasetcsv = spark.read.option("header", true)
.option("multiline",true).csv("/home/martinez/git/ms2020bgd/SD701/airbnb/airbnb_paris.csv")

datasetcsv.select("host_about").show(false)


root
 |-- value: string (nullable = true)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|host_about                                                                                                                                                                                                                                                                                                                                                                                                                      

import spark.implicits._
dataset: org.apache.spark.sql.DataFrame = [value: string]
toSave: org.apache.spark.sql.DataFrame = [value: string]
datasetcsv: org.apache.spark.sql.DataFrame = [id: string, listing_url: string ... 104 more fields]


In [3]:
toSave.coalesce(1).write.text("/home/martinez/git/ms2020bgd/SD701/airbnb/airbnb_paris2.csv")

org.apache.spark.sql.AnalysisException:  path file:/home/martinez/git/ms2020bgd/SD701/airbnb/airbnb_paris2.csv already exists.;

In [4]:
val datasetNew = spark.read.option("header", true)
.option("multiline",true).csv("/home/martinez/git/ms2020bgd/SD701/airbnb/airbnb_paris2.csv")

datasetNew: org.apache.spark.sql.DataFrame = [id: string, listing_url: string ... 104 more fields]


In [5]:
datasetNew.printSchema()
datasetNew.select("interaction").show(false)

root
 |-- id: string (nullable = true)
 |-- listing_url: string (nullable = true)
 |-- scrape_id: string (nullable = true)
 |-- last_scraped: string (nullable = true)
 |-- name: string (nullable = true)
 |-- summary: string (nullable = true)
 |-- space: string (nullable = true)
 |-- description: string (nullable = true)
 |-- experiences_offered: string (nullable = true)
 |-- neighborhood_overview: string (nullable = true)
 |-- notes: string (nullable = true)
 |-- transit: string (nullable = true)
 |-- access: string (nullable = true)
 |-- interaction: string (nullable = true)
 |-- house_rules: string (nullable = true)
 |-- thumbnail_url: string (nullable = true)
 |-- medium_url: string (nullable = true)
 |-- picture_url: string (nullable = true)
 |-- xl_picture_url: string (nullable = true)
 |-- host_id: string (nullable = true)
 |-- host_url: string (nullable = true)
 |-- host_name: string (nullable = true)
 |-- host_since: string (nullable = true)
 |-- host_location: string (nullable

In [31]:
// On recupere les colonnes qui nous interessent

val datasetTypes = datasetNew.withColumn("accommodates", $"accommodates".cast("Int")).withColumn("accommodates", when($"accommodates".isNull, 2).otherwise($"accommodates"))
                         .withColumn("bathrooms", $"bathrooms".cast("Int")).withColumn("bathrooms", when($"bathrooms".isNull, 0).otherwise($"bathrooms"))
                         .withColumn("guests_included", $"guests_included".cast("Int")).withColumn("guests_included", when($"guests_included".isNull, 0).otherwise($"guests_included"))
                         .withColumn("bedrooms", $"bedrooms".cast("Int")).withColumn("bedrooms", when($"bedrooms".isNull, 0).otherwise($"bedrooms"))
                         .withColumn("beds", $"beds".cast("Int")).withColumn("beds", when($"beds".isNull, 0).otherwise($"beds"))
                         .withColumn("price",regexp_replace($"price","[$]","")).withColumn("price", $"price".cast("Double"))
                         .withColumn("cleaning_fee",regexp_replace($"cleaning_fee","[$]","")).withColumn("cleaning_fee", $"cleaning_fee".cast("Double")).withColumn("cleaning_fee", when($"cleaning_fee".isNull, 0).otherwise($"cleaning_fee"))
                         .withColumn("name", when($"name".isNull, "").otherwise($"name"))
                         .withColumn("summary", when($"summary".isNull, "").otherwise($"summary"))
                         .withColumn("space", when($"space".isNull, "").otherwise($"space"))
                         .withColumn("description", when($"description".isNull, "").otherwise($"description"))
                         .withColumn("neighborhood_overview", when($"neighborhood_overview".isNull, "").otherwise($"neighborhood_overview"))
                         .withColumn("text", concat_ws(" ", $"name", $"summary", $"space", $"description",$"neighborhood_overview"))
                         .withColumn("amenities", regexp_replace($"amenities","[{|}]","")).withColumn("amenities", split($"amenities", ","))
                         .withColumn("room_type", when($"room_type".isNull, "unknown").otherwise($"room_type"))
                         .withColumn("property_type", when($"property_type".isNull, "unknown").otherwise($"property_type"))
                         .withColumn("neighbourhood", when($"neighbourhood".isNull, "unknown").otherwise($"neighbourhood"))


val subDataSet = datasetTypes.select("accommodates","bathrooms","guests_included","bedrooms","beds",
                                     "price", "cleaning_fee","bedrooms", "text", "property_type","room_type","neighbourhood", "review_scores_rating","amenities")
                        .withColumn("review_scores_rating", $"review_scores_rating".cast("Double"))
                        .filter(!$"review_scores_rating".isNull).filter(!$"price".isNull)






datasetTypes: org.apache.spark.sql.DataFrame = [id: string, listing_url: string ... 105 more fields]
subDataSet: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [accommodates: int, bathrooms: int ... 12 more fields]


In [32]:
// Get distinct tags array
val amenities = subDataSet
  .flatMap(r ⇒ r.getAs[Seq[String]]("amenities"))
  .distinct()
  .collect()
  .sortWith(_ < _)

val cvmData = new CountVectorizerModel(amenities)
  .setInputCol("amenities")
  .setOutputCol("sparseAmenities")
  .transform(subDataSet)

val asDense = udf((v: Vector) ⇒ v.toDense)

val dataSetFull = cvmData.withColumn("features", asDense($"sparseAmenities"))

amenities: Array[String] = Array("", " toilet", 24-hour check-in, Accessible-height bed, Accessible-height toilet, Air conditioning, Air purifier, Amazon Echo, BBQ grill, Baby bath, Baby monitor, Babysitter recommendations, Balcony, Bath towel, Bathroom essentials, Bathtub, Bathtub with bath chair, Beach essentials, Beachfront, Bed linens, Bedroom comforts, Bidet, Body soap, Breakfast, Breakfast table, Building staff, Buzzer/wireless intercom, Cable TV, Carbon monoxide detector, Cat(s), Ceiling fan, Ceiling hoist, Central air conditioning, Changing table, Children’s books and toys, Children’s dinnerware, Cleaning before checkout, Coffee maker, Convection oven, Cooking basics, Crib, DVD player, Day bed, Disabled parking spot, Dishes and silverware, Dishwasher, Dog(s), Doorman, Double ove...

In [ ]:




// Let's start the pipeline

val indexerRoomType = new StringIndexer().setInputCol("room_type").setOutputCol("room_index").setHandleInvalid("skip")
val indexerPropertyType = new StringIndexer().setInputCol("property_type").setOutputCol("property_index").setHandleInvalid("skip")
val indexerNeighbourhoudType = new StringIndexer().setInputCol("neighbourhood").setOutputCol("neighbourhood_index").setHandleInvalid("skip")

val oneHotEncorderCountry = new OneHotEncoderEstimator().setDropLast(false).setInputCols(Array(indexerRoomType.getOutputCol, indexerPropertyType.getOutputCol, indexerNeighbourhoudType.getOutputCol))
    .setOutputCols(Array("room_onehot", "property_onehot", "neighbourhood_onehot"))


val tokenizer = new RegexTokenizer().setPattern("\\W+").setGaps(true).setInputCol("text").setOutputCol("tokens")
val stopWordRemover = new StopWordsRemover().setInputCol(tokenizer.getOutputCol).setOutputCol("filtered")


val countVectorizer = new CountVectorizer().setInputCol(stopWordRemover.getOutputCol).setOutputCol("TF").setMinDF(50)


  //    val dataSetHashing = new HashingTF().setInputCol("filtered").setOutputCol("features").transform(dataSetCountVect)
  //
val IDF = new IDF().setInputCol(countVectorizer.getOutputCol).setOutputCol("tfidf")

val vectorAssembler = new VectorAssembler().setInputCols(Array(
    "tfidf",
    "room_onehot", "property_onehot", "neighbourhood_onehot", "features",
                                                                "accommodates","bathrooms","guests_included","bedrooms","beds",
                                                                "price", "cleaning_fee"
                                                              )).setOutputCol("features_assembled")

val rf = new RandomForestRegressor().setLabelCol("review_scores_rating").setFeaturesCol("features_assembled")

val lr = new LinearRegression()
   .setLabelCol("review_scores_rating")
   .setFitIntercept(true)
  .setFeaturesCol("features_assembled")
.setRegParam(0.2)
  .setMaxIter(100)

val pipeline: Pipeline = new Pipeline().setStages(Array(tokenizer, stopWordRemover, countVectorizer, IDF
                                                        , indexerRoomType, indexerPropertyType,indexerNeighbourhoudType, oneHotEncorderCountry, vectorAssembler, rf))

val Array(training, test) = dataSetFull.randomSplit(Array(0.9, 0.1), 999)

val model = pipeline.fit(training)
val dfWithSimplePredictions = model.transform(test)

val evaluator = new RegressionEvaluator()
  .setLabelCol("review_scores_rating")
  .setPredictionCol("prediction")
  .setMetricName("rmse")

val rmse = evaluator.evaluate(dfWithSimplePredictions)
println(s"Root Mean Squared Error (RMSE) on test data = $rmse")




In [14]:
dfWithSimplePredictions.select("tfidf").show(100, false)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------